In [1]:
import os
import sys, glob
import argparse
# 異なる階層のutilsをインポートするために必要
sys.path.append("../")
import numpy as np
import math
import re
import random
from collections import defaultdict
# 異なる階層のutilsからインポート
from utils.constant import *
from utils.help_func import load_pickle, save_pickle, load_json, save_json
# 異なる階層のmodel_utilsからインポート
from model_utils.model_util import load_model, sent2tensor
from model_utils import train_args


In [36]:
B = 10
variants = ['srnn', 'gru', 'lstm']
thetas = [10, 50]
method_names = ['ochiai', 'tarantula', 'dstar', 'ochiai2', 'ample']
num_dataset = 8
k_list, n_list = list(range(2, 12, 2)), list(range(1, 6, 1))
num_k, num_n = len(k_list), len(n_list)

for bi in range(B):
  for method in method_names:
    save_path = f'./artifacts/pafl_res_fortest/boot_{bi}/{method}_pafl_res.csv'
    res_list = []
    for model_type in variants:
      for theta in thetas:      
        dataset_res_per_n = np.zeros((num_n, num_dataset, 6))
        for ni, n in enumerate(n_list):
          load_path = get_path(f'data/extracted_data/{model_type}/pred_table_for_look/{method}/boot_{bi}/n={n}_theta={theta}.csv')
          pafl_res = np.genfromtxt(load_path, delimiter=',', dtype='float')[1:, 2:]
          pafl_res_ex = pafl_res[:,::2].astype(np.float32) # loadした直後のshape: (40, 12)を[:,::2]で(40, 6)にする(ランダムのを落とすため)
          # boot_iのファイルではmnist->toxicの順版になってるのでこれを入れ替える
          # そのために，pafl_res_exの25-32行目と33-40行目を入れ替える
          order  = list(range(24)) +  list(range(32, 40)) + list(range(24, 32))
          pafl_res_ex = pafl_res_ex[order, :]
          for dataset_id, i in enumerate(range(0, num_dataset*num_k, num_k)):
            dataset_res = pafl_res_ex[i:i+num_k, :]
            # print(dataset_res.shape) #(num_k, 6)
            dataset_res = np.where(dataset_res < 0, np.nan, dataset_res)
            # theta, n, datasetを固定したときの全k(5通り)の中の最小値を取る
            dataset_res_k_aggregated = np.nanmin(dataset_res, axis=0)
            # print(dataset_res_k_aggregated.shape) #(6, )
            dataset_res_per_n[ni][dataset_id] = dataset_res_k_aggregated
        # theta,datasetを固定したときの全n, kの中の最小値を取る
        pafl_res_aggregated = np.nanmin(dataset_res_per_n, axis=0)
        # print(pafl_res_aggregated.shape) # (8, 6) = (#datasets, #metrics)
        res_list.append(pafl_res_aggregated)
    # print(len(res_list)) # 6(=2*3), 6つの各要素が(8,6)の配列
    res_arr = np.concatenate(res_list)
    # print(res_arr.shape) # (48, 6)
    # 保存パスを指定して保存
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    np.savetxt(save_path, res_arr, delimiter=',', fmt='%.3f')
    print(f'saved in {save_path}')

saved in ./artifacts/pafl_res_fortest/boot_0/ochiai_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_0/tarantula_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_0/dstar_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_0/ochiai2_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_0/ample_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_1/ochiai_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_1/tarantula_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_1/dstar_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_1/ochiai2_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_1/ample_pafl_res.csv


/tmp/ipykernel_1618/137053920.py:29: RuntimeWarning: All-NaN slice encountered
  dataset_res_k_aggregated = np.nanmin(dataset_res, axis=0)
/tmp/ipykernel_1618/137053920.py:33: RuntimeWarning: All-NaN slice encountered
  pafl_res_aggregated = np.nanmin(dataset_res_per_n, axis=0)


saved in ./artifacts/pafl_res_fortest/boot_2/ochiai_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_2/tarantula_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_2/dstar_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_2/ochiai2_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_2/ample_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_3/ochiai_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_3/tarantula_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_3/dstar_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_3/ochiai2_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_3/ample_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_4/ochiai_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_4/tarantula_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_4/dstar_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_4/ochiai2_pafl_res.csv
saved in ./artifacts/pafl_res_fortest/boot_4/ample_pafl_res.csv
saved in ./artifact